In [2]:
import scipy
import scipy.io
import matplotlib.pyplot as plt
import glob
import json
import math
import os
import subprocess
import tempfile
import numpy as np
import regex
from scipy.sparse import csr_matrix

# Produce rhs

In [6]:
work_folder="/home/yiwei/results_new/test_refine_0.00/2d_bar/AMGCL/square_beam2d_structed/bar_2d/P1/ref0/block2/Thread32/0/output"
stiffness=scipy.io.mmread(work_folder+"/stiffness.mat")
solution=np.loadtxt(work_folder+"/solution.mat")
rhs=(stiffness.tocsr()).dot(solution)
rhs=np.reshape(rhs,(rhs.shape[0],1))
scipy.io.mmwrite(work_folder+"/rhs.mtx",rhs)

# Produce new matrix and new rhs

In [7]:
stiff_triu=scipy.sparse.triu(stiffness,k=0,format="csr")
disjoint_id=[]
disjoint_value=[]
disjoint_ptr=[]
mask_val = np.ones_like(stiff_triu.data,dtype=np.bool8)
for i in range(len(stiff_triu.indptr)-1):    
    if stiff_triu.data[stiff_triu.indptr[i]]==1.0:
        disjoint_id.append(i)
        disjoint_value.append(stiff_triu.data[stiff_triu.indptr[i]])
        mask_val[stiff_triu.indptr[i]]=False
        disjoint_ptr.append(stiff_triu.indptr[i])
# disjoint_ptr.append(len(stiff_triu.indices))
mask_ptr = np.array([(i-1 in disjoint_id) for i in range(len(stiff_triu.indptr))])
tmp_indptr=stiff_triu.indptr.copy()
tmp_indices=stiff_triu.indices.copy()
for i in range(len(disjoint_ptr)):
    tmp_indptr[disjoint_id[i]+1:]=tmp_indptr[disjoint_id[i]+1:]-1
for i in range(len(tmp_indices)):
    indice=tmp_indices[i]
    k=0
    for id in disjoint_id:
        if indice>id:
            k=k+1
    tmp_indices[i]=tmp_indices[i]-k
new_triu=csr_matrix((stiff_triu.data[mask_val], tmp_indices[mask_val], tmp_indptr[~mask_ptr]),(120,120))
new_stiff=new_triu+new_triu.transpose()-scipy.sparse.diags(new_triu.diagonal())
new_rhs=rhs[~(np.array([(i in disjoint_id) for i in range(len(rhs))]))]
new_folder=work_folder+"/new_matrix"
if (not os.path.exists(new_folder)):
    os.makedirs(new_folder)
scipy.io.mmwrite(new_folder+"/rhs.mtx",new_rhs)
scipy.io.mmwrite(new_folder+"/stiffness.mat",new_stiff,symmetry="general")

# Run ML

In [12]:
work_folder="/home/yiwei/results_new/test_refine_0.00/2d_bar/AMGCL/square_beam2d_structed/bar_2d/P1/ref0/block2/Thread32/0/output/new_matrix"
ml_exe="/home/yiwei/Trilinos/build/packages/ml/examples/BasicExamples/ML_read_MatrixMarket.exe"
matrix=work_folder+"/stiffness.mat"
rhs=work_folder+"/rhs.mtx"
block_size=2
solver="cg"
output_folder=work_folder
logname='/log_ml_'+str(block_size)+'.txt'
if (not os.path.exists(output_folder)):
    os.makedirs(output_folder)
logfile = open(output_folder+logname, "w")
os.chdir(output_folder)
args = [ml_exe,
"--matrix="+matrix,"--krylov="+solver,"--npdes="+str(block_size),"--rhs="+rhs]
subprocess.run(args,stdout=logfile,stderr=logfile) 
logfile.close()

# Print Hierachy

In [13]:
logfile=open(output_folder+logname, "r")
hierachy=[]
rows=[]
unknowns=[]
iterations=-1
for line in logfile.readlines():
    m=regex.match(r"Smoother \(level \d\) : # global rows = \d+",line)
    n=regex.search(r"total iterations: \d+",line)
    if m!=None:
        tmp=regex.findall(r"\d+",line)
        hierachy.append(int(tmp[0]))
        rows.append(int(tmp[1]))
        unknowns.append(int(tmp[2]))
    if n!=None:
        tmp=regex.findall(r"\d+",n.group(0))   
        iterations=int(tmp[0])
for hie,unk,row in zip(hierachy,unknowns,rows):
    print("Level ",hie," Global rows ",row," Unknowns ", unk)
print("Iterations",iterations)

Level  0  Global rows  120  Unknowns  1250
Level  1  Global rows  16  Unknowns  180
Iterations 76
